In [7]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [8]:
from src.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from src.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector
from src.bias.FreqVsRefBiasDetector import FreqVsRefBiasDetector

## Importing Data and Trained Classifier

**UX**: 
- the user uploads the dataset with the features (X)
- the user uploads 
    - the sklearn model as pkl file **or**
    - a file with the predictions already created (Y). 
- the user uploads scaler and ohe (if needed)

If a model is provided, the application checks if the the provided datasets and model match in terms of column names. 

In [9]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [10]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

In [11]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [12]:
with open("trained_model_for_testing/RF_12_200.pkl", "rb") as file:
    classifier = load(file)

/home/dev/repos/venv_gen_purpose/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dev/repos/venv_gen_purpose/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
predicted_prob = classifier.predict_proba(X_test_ohe)
predicted_values = classifier.predict(X_test_ohe)

#### Definition of conditioning variables
**UX**:
- the user selects the continuous features to be categorized and used for the creation of control groups. 

The application propose splitting cuts that make the resulting discrete distribution uniform (but also other binning procedures are possible). 

In [14]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

**UX**:
- the user selects from a drop down menu the variables to be used for the groups creation
- the user can select "check all the variables" and the application will use all the available discrete features

In [15]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [16]:
df_with_predictions = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values)], axis=1).rename(columns={0:"predictions"})

## Bias Detection

**UX**:
- the user selects from a drop down menu the binary variable to be used for the distance calculation
- the user selects a threshold for the distance
- the user selects a Distance (if more than one are provided)

In [17]:
bd = FreqVsFreqBiasDetector(distance="TVD")

**UX**:

The user has can select between two options:
- Option 1: distance between frequencies
- Option 2: distance from reference distribution
    - in this case, the user has to insert values for a reference distribution

## Option 1: Distance between frequencies

### Comparison of the two groups observed freqs

In [18]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    threshold=0.1
)

(0.025269625352224545, True, 0.1, None)

In [19]:
#distance = max( abs(female_0_freq - male_0_freq), abs(female_1_freq - male_1_freq) )

In [20]:
df_with_predictions.groupby("x2_sex").predictions.value_counts(normalize=True)

x2_sex  predictions
1       0              0.873592
        1              0.126408
2       0              0.898862
        1              0.101138
Name: predictions, dtype: float64

### Comparison of the two groups observed freqs conditioning to other features

In [21]:
results = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30)

In [22]:
results

{'x3_education==1': (3119, 0.0183300648997351, True, 0.1, None),
 'x3_education==3': (1499, 0.03290896164530149, True, 0.1, None),
 'x3_education==2': (4250, 0.030006620324395897, True, 0.1, None),
 'x3_education==4': (40, 0.0, True, 0.1, None),
 'x3_education==5': (75, 0.06772575250836121, True, 0.1, None),
 'x3_education==6': (14, None, 'Not enough observations'),
 'x3_education==0': (3, None, 'Not enough observations'),
 'x4_marriage==1': (4065, 0.019241036358938257, True, 0.1, None),
 'x4_marriage==2': (4822, 0.03149828894509746, True, 0.1, None),
 'x4_marriage==3': (95, 0.05425219941348974, True, 0.1, None),
 'x4_marriage==0': (18, None, 'Not enough observations'),
 'age_buckets==3': (2727, 0.01788279719173691, True, 0.1, None),
 'age_buckets==1': (2895, 0.009266981744045971, True, 0.1, None),
 'age_buckets==2': (3378, 0.047657731623916205, True, 0.1, None),
 'x3_education==1 & x4_marriage==1': (1106,
  0.02132550290983498,
  True,
  0.1,
  None),
 'x3_education==1 & x4_marriage==

**UX**:
- the application shows the results in two views:
    - overall results: all the computed distances
    - violations: only the results above the threshold
- for both, the user can order the results by number of obs, distance or group name

In [23]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if not v[2]}

- '2: Gender (1 = male; 2 = female).',
- '3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).',
- '4: Marital status (1 = married; 2 = single; 3 = others).'

In [24]:
# sorting the violations by number of observations belonging to that group
dict(sorted(violations.items(), key=lambda item: item[1], reverse=True))

{'x3_education==3 & x4_marriage==2 & age_buckets==2': (189,
  0.10919540229885057,
  False,
  0.1,
  None)}

#### Focus analyses on particular cases
Here we want to understand/qualify some specific violation cases.

**UX**: 
- the user selects one of the available violations
- the application returns relevant details to help understand what's going on
- the user can download the results as csv file. 

In [25]:
focus_df = df_with_predictions.query("x3_education==3 & x4_marriage==2 & age_buckets==2")
bd.get_frequencies_list(focus_df, 'predictions', 
                        df_with_predictions.predictions.unique(),
                        'x2_sex', df_with_predictions.x2_sex.unique())

([array([0.94252874, 0.05747126]), array([0.83333333, 0.16666667])],
 [array([82,  5]), array([85, 17])])

In [26]:
focus_df.groupby('x2_sex').predictions.value_counts(normalize=True)

x2_sex  predictions
1       0              0.833333
        1              0.166667
2       0              0.942529
        1              0.057471
Name: predictions, dtype: float64

## Option 2: distance from reference distribution

In [27]:
bd = FreqVsRefBiasDetector()

In [34]:
male_0_ref = 75/100
male_1_ref = 25/100

female_0_ref = 75/100
female_1_ref = 25/100

ref_distribution = [np.array([female_0_ref, female_1_ref]), np.array([male_0_ref, male_1_ref])]

### Comparison of the two groups w.r.t. the reference distribution

In [35]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    threshold=0.1,
    reference_distribution=ref_distribution
)

([0.13069171206061392, 0.17902898484562713], [False, False], 0.1)

### Comparison of the two groups w.r.t. the reference distribution conditioning to other features

In [30]:
results = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30,
    reference_distribution=ref_distribution)

In [31]:
results

{'x3_education==1': (3119,
  [0.15292885987559768, 0.19953637846872263],
  [False, False],
  0.1),
 'x3_education==3': (1499,
  [0.03232914493273209, 0.06683511671201015],
  [True, True],
  0.1),
 'x3_education==2': (4250,
  [0.04817486426708861, 0.08642530432448603],
  [True, True],
  0.1),
 'x3_education==4': (40, [1.0, 1.0], [False, False], 0.1),
 'x3_education==5': (75,
  [0.14386916131129301, 0.38832031424465063],
  [False, False],
  0.1),
 'x3_education==6': (14, None, 'Not enough observations'),
 'x3_education==0': (3, None, 'Not enough observations'),
 'x4_marriage==1': (4065,
  [0.07892104655259335, 0.10973782259615394],
  [True, False],
  0.1),
 'x4_marriage==2': (4822,
  [0.07082922204298259, 0.12158030611282034],
  [True, False],
  0.1),
 'x4_marriage==3': (95,
  [0.13547714823554813, 0.0516801997479408],
  [False, True],
  0.1),
 'x4_marriage==0': (18, None, 'Not enough observations'),
 'age_buckets==3': (2727,
  [0.07325935937033368, 0.10037160327202987],
  [True, False],

In [32]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if (not v[2][0] or not v[2][1])}

In [33]:
violations

{'x3_education==1': (3119,
  [0.15292885987559768, 0.19953637846872263],
  [False, False],
  0.1),
 'x3_education==4': (40, [1.0, 1.0], [False, False], 0.1),
 'x3_education==5': (75,
  [0.14386916131129301, 0.38832031424465063],
  [False, False],
  0.1),
 'x4_marriage==1': (4065,
  [0.07892104655259335, 0.10973782259615394],
  [True, False],
  0.1),
 'x4_marriage==2': (4822,
  [0.07082922204298259, 0.12158030611282034],
  [True, False],
  0.1),
 'x4_marriage==3': (95,
  [0.13547714823554813, 0.0516801997479408],
  [False, True],
  0.1),
 'age_buckets==3': (2727,
  [0.07325935937033368, 0.10037160327202987],
  [True, False],
  0.1),
 'age_buckets==2': (3378,
  [0.08024299977384175, 0.17187679217619678],
  [True, False],
  0.1),
 'x3_education==1 & x4_marriage==1': (1106,
  [0.1732719966909978, 0.23477269281627888],
  [False, False],
  0.1),
 'x3_education==1 & x4_marriage==2': (2002,
  [0.14048573698048983, 0.18238653638283597],
  [False, False],
  0.1),
 'x3_education==5 & x4_marriage=